In [109]:
import pathlib
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle


In [21]:
main_dir=pathlib.Path(".")

In [107]:
data={
    "mutation":[],
    "agent":[],
    "ko":[],
    "reward":[]
}
sample_num=50
fig=go.Figure()
for ko in main_dir.iterdir():
    if ko.name.startswith(".") or ko.name.startswith("Data_Analysis.ipynb"):
        continue
    else: 
        agent1_ko,agent2_ko=ko.name.split("_")
        if (ko/"returns_1800.json").exists():
            with open(ko/"returns_1800.json") as f:
                rewards=json.load(f)
            data["reward"].extend(rewards["agent1"][-sample_num:])
            data["reward"].extend(rewards["agent2"][-sample_num:])
            data["mutation"].extend([agent1_ko]*sample_num+[agent2_ko]*sample_num)
            data["ko"].extend([ko.name]*sample_num*2)
            data["agent"].extend(["agent1"]*sample_num+["agent2"]*sample_num)
                
            
                        
            
        
data=pd.DataFrame(data)    
        

In [108]:
fig.add_trace(go.Box(
    x=data[data["agent"]=="agent1"]["ko"],
    y=data[data["agent"]=="agent1"]["reward"]
))
fig.add_trace(go.Box(
    x=data[data["agent"]=="agent2"]["ko"],
    y=data[data["agent"]=="agent2"]["reward"]
))
fig.update_layout(boxmode="group",width=800)


In [ ]:
with open()